## Part 2: FourSquare and Yelp APIs
This notebook makes use of the City Bikes data collected for Part 1. It is loaded from a CSV file before accesing the APIs.

In [39]:
# import required libraries
import pandas as pd
import os # use this to access environment variables
import requests # this will be used to call the APIs
import json

# get the Foursquare environment variable from the OS file
FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')

# they city_bikes.ipnb notebook created a CSV for it's output, and this is read into a dataframe here
city_bikes_df = pd.read_csv('../data/city_bikes.csv')
city_bikes_df.head()

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra.uid,extra.number,extra.slots,extra.bike_uids,extra.virtual
0,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,2025-07-01T19:18:19.376003+00:00Z,11,0,19738176,8416,6,"['47041', '823506', '46976', '46961', '46910',...",False
1,0a106cbc148d5a0c2535b51c1dbe3b4d,Parkhead,55.846524,-4.197475,2025-07-01T19:18:19.377604+00:00Z,6,0,150125231,8470,8,"['82316', '840047', '46722', '46457', '47968',...",False
2,0cc8c09950e1435ee7782478ed292fef,Tollcross International Swimming Centre,55.844250,-4.176167,2025-07-01T19:18:19.376951+00:00Z,6,0,55599921,8200,6,"['828253', '46967', '823632', '828964', '82360...",False
3,0e94d2ad012bff0cf23497963ff5fd77,Shields Road,55.845807,-4.275232,2025-07-01T19:18:19.375677+00:00Z,1,7,7145152,8403,8,['46992'],False
4,101c6cd7749f373507e9444f5222b7f2,Shawlands Shopping Centre (Kilmarnock Road),55.829057,-4.282675,2025-07-01T19:18:19.375152+00:00Z,1,6,3812605,8453,8,['46830'],False


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [52]:
def get_venues_fs(latitude, longitude, radius, api_key, categories=None):
    """
    Get venues from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        number of venues (POI) of the specified category
    """
    
    url = "https://places-api.foursquare.com/places/search"
    
    params = {
        "ll":f"{latitude},{longitude}",
        "radius": radius,
        "limit": 50
    }

    if categories:
        params["categories"] = categories

    headers = {
        "accept": "application/json",
        "X-Places-Api-Version": "2025-06-17",
        "authorization": "Bearer "+api_key
    }

    response = requests.get(url, params=params, headers=headers)

    # print(response.text)
    # print(json.loads(response.text))
    # print(response.json().get("results", []))
    
    # The function returns the venues for the specified area
    return response.json().get("results", [])


# Fixed parameters
API_KEY = os.getenv('FOURSQUARE_API_KEY')
RADIUS = 1000  # meters
CATEGORIES = "4bf58dd8d48988d116941735" # The category code for bars and similar businesses. 
# I'm not sure that this is working correctly, as there appear to be a lot of entries that are likely not bars.
# It could just be that I've chosen a FourSquare category that's broad and non-specific.

# Call the function for each row. 
# This adds the information retrieved from the Foursquare API to a dataframe.

fs_df = pd.DataFrame()

for i in range(0,1):   #len(city_bikes_df)):
    print(i)
    lat = city_bikes_df['latitude'][i]
    lon = city_bikes_df['longitude'][i]
    id = city_bikes_df['id'][i]

    fs_json = get_venues_fs(
        latitude= lat,
        longitude= lon,
        radius=RADIUS,
        api_key=API_KEY,
        categories=CATEGORIES
    )
    print(fs_json)

    fs_venues_df = pd.json_normalize(
        fs_json,
        record_path=['categories'],
        meta=[
            'fsq_place_id',
            'latitude',
            'longitude',
            'distance',
            'email',
            'link',
            ['location', 'address'],
            ['location', 'locality'],
            ['location', 'region'],
            ['location', 'postcode']
        ],
        errors = 'ignore'
    )
    
    fs_df = pd.concat([fs_df,fs_venues_df])
    fs_df['bike_lat'] = lat
    fs_df['bike_lon'] = lon
    fs_df['bike_id'] = id
fs_df


# city_bikes_df['fs_venues'] = city_bikes_df.apply(
#     lambda row: get_venues_fs(
#         latitude=row['latitude'],
#         longitude=row['longitude'],
#         radius=RADIUS,
#         api_key=API_KEY,
#         categories=CATEGORIES
#     ),
#     axis=1  # Apply to each row
# )

# city_bikes_df


0
[{'fsq_place_id': '4c8a3ba4a92fa093be438fbf', 'latitude': 55.86539237825414, 'longitude': -4.3064174398355215, 'categories': [{'fsq_category_id': '4bf58dd8d48988d181941735', 'name': 'Museum', 'short_name': 'Museum', 'plural_name': 'Museums', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_', 'suffix': '.png'}}, {'fsq_category_id': '4bf58dd8d48988d190941735', 'name': 'History Museum', 'short_name': 'History Museum', 'plural_name': 'History Museums', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_history_', 'suffix': '.png'}}], 'date_created': '2010-09-10', 'date_refreshed': '2025-06-30', 'distance': 73, 'email': 'museums@glasgowlife.org.uk', 'extended_location': {}, 'link': '/places/4c8a3ba4a92fa093be438fbf', 'location': {'address': 'Glasgow Harbour', 'locality': 'Glasgow', 'region': 'Glasgow City', 'postcode': 'G11 6AR', 'admin_region': 'Scotland', 'country': 'GB', 'formatted_address': 'Glasgow Harbour (100 Po

,fsq_category_id,name,short_name,plural_name,icon.prefix,icon.suffix,fsq_place_id,latitude,longitude,distance,email,link,location.address,location.locality,location.region,location.postcode,bike_lat,bike_lon,bike_id
0,4bf58dd8d48988d181941735,Museum,Museum,Museums,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4c8a3ba4a92fa093be438fbf,55.865392,-4.306417,73,museums@glasgowlife.org.uk,/places/4c8a3ba4a92fa093be438fbf,Glasgow Harbour,Glasgow,Glasgow City,G11 6AR,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
1,4bf58dd8d48988d190941735,History Museum,History Museum,History Museums,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4c8a3ba4a92fa093be438fbf,55.865392,-4.306417,73,museums@glasgowlife.org.uk,/places/4c8a3ba4a92fa093be438fbf,Glasgow Harbour,Glasgow,Glasgow City,G11 6AR,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
2,4bf58dd8d48988d16d941735,Café,Café,Cafés,https://ss3.4sqi.net/img/categories_v2/food/cafe_,.png,4eaaa5f477c850207ed9fee8,55.86561,-4.305612,17,NaN,/places/4eaaa5f477c850207ed9fee8,Riverside Museum,Glasgow,Glasgow City,,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
3,4bf58dd8d48988d1e2931735,Art Gallery,Art Gallery,Art Galleries,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4b86d7fcf964a52018a131e3,55.864562,-4.299637,378,work@swg3.tv,/places/4b86d7fcf964a52018a131e3,100 Eastvale Pl.,Glasgow,Glasgow City,G3 8QG,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
4,4bf58dd8d48988d1e5931735,Music Venue,Music Venue,Music Venues,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4b86d7fcf964a52018a131e3,55.864562,-4.299637,378,work@swg3.tv,/places/4b86d7fcf964a52018a131e3,100 Eastvale Pl.,Glasgow,Glasgow City,G3 8QG,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,4bf58dd8d48988d16d941735,Café,Café,Cafés,https://ss3.4sqi.net/img/categories_v2/food/cafe_,.png,4bba108998c7ef3b62a33202,55.867384,-4.291243,900,NaN,/places/4bba108998c7ef3b62a33202,,Glasgow,Glasgow City,G3 8AF,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
58,4bf58dd8d48988d1c5941735,Sandwich Spot,Sandwich Spot,Sandwich Spots,https://ss3.4sqi.net/img/categories_v2/food/deli_,.png,50868fdce4b0fedcb3c523e5,55.86656,-4.289764,977,NaN,/places/50868fdce4b0fedcb3c523e5,1299 Argyle St,Glasgow,Glasgow City,G3 8TL,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
59,4bf58dd8d48988d1e0931735,Coffee Shop,Coffee Shop,Coffee Shops,https://ss3.4sqi.net/img/categories_v2/food/co...,.png,50868fdce4b0fedcb3c523e5,55.86656,-4.289764,977,NaN,/places/50868fdce4b0fedcb3c523e5,1299 Argyle St,Glasgow,Glasgow City,G3 8TL,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4
60,4bf58dd8d48988d116941735,Bar,Bar,Bars,https://ss3.4sqi.net/img/categories_v2/nightli...,.png,4d135e2f37f1236a1cc2801b,55.870855,-4.302688,598,NaN,/places/4d135e2f37f1236a1cc2801b,232 Dumbarton Rd.,Glasgow,NaN,G11 6TU,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4


In [ ]:
# Show an example of the FourSquare data that was saved into the fs_venues
city_bikes_df['fs_venues'][0]

[{'fsq_place_id': '4c8a3ba4a92fa093be438fbf',
  'latitude': 55.86539237825414,
  'longitude': -4.3064174398355215,
  'categories': [{'fsq_category_id': '4bf58dd8d48988d181941735',
    'name': 'Museum',
    'short_name': 'Museum',
    'plural_name': 'Museums',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_',
     'suffix': '.png'}},
   {'fsq_category_id': '4bf58dd8d48988d190941735',
    'name': 'History Museum',
    'short_name': 'History Museum',
    'plural_name': 'History Museums',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_history_',
     'suffix': '.png'}}],
  'date_created': '2010-09-10',
  'date_refreshed': '2025-06-29',
  'distance': 73,
  'email': 'museums@glasgowlife.org.uk',
  'extended_location': {},
  'link': '/places/4c8a3ba4a92fa093be438fbf',
  'location': {'address': 'Glasgow Harbour',
   'locality': 'Glasgow',
   'region': 'Glasgow City',
   'postcode': 'G11 6AR',
   'admin_region':

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [ ]:
# flatten the json returned by the FourSquare API and stored in the city_bikes_df dataframe
fs_venues_df = pd.json_normalize(
    city_bikes_df['fs_venues'][0],
    record_path=['categories'],
    meta=[
        'fsq_place_id',
        'latitude',
        'longitude',
        'distance',
        'email',
        'link',
        ['location', 'address'],
        ['location', 'locality'],
        ['location', 'region'],
        ['location', 'postcode']
    ],
    errors = 'ignore'
)
fs_venues_df

KeyError: 'id'

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests

def get_venues_yelp(latitude, longitude, radius, api_key):
    """
    Get venues from Yelp within a specified location.
    Args:
        latitude (float): latitude for query
        longitude (float): longitude for query
        radius (int): how far to search from central point
        api_key (str): Yelp API key to use for query
        
    Returns:
        venues (POI) information
    """
    
    url = "https://api.yelp.com/v3/businesses/search?"
    # Full query example "https://api.yelp.com/v3/businesses/search?latitude=55.865392&longitude=-4.306417&radius=1000&sort_by=best_match&limit=20"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "sort_by": 'best_match',
        "limit": 20  #A limit placed on how many results to return for each request.
    }

    headers = {
        "accept": "application/json",
        "authorization": f"Bearer {api_key}"
    }

    response = requests.get(url, params=params, headers=headers) # Make the appropriate request to Yelp using the above params/headers.
    response.raise_for_status()  # Check for HTTP errors

    # print(response.text)
    # print(json.loads(response.text))
    # print(response.json().get("businesses", []))
    
    # The function returns the venues for the specified area
    return response.json().get("businesses", [])


# Fixed parameters
YELP_KEY = os.getenv('YELP_KEY')  # get the YELP environment variable from the OS file
RADIUS = 1000  # meters

# Apply the function row-wise
city_bikes_df['yelp_venues'] = city_bikes_df.apply(
    lambda row: get_venues_yelp(
        latitude=row['latitude'],
        longitude=row['longitude'],
        radius=RADIUS,
        api_key=YELP_KEY
    ),
    axis=1  # Apply to each row
)

city_bikes_df


[{'id': '_GHNOOJhhjLegiCvzf5V0Q', 'alias': 'ox-and-finch-glasgow', 'name': 'Ox And Finch', 'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/nro4UcJrJkLiPOQ8Yvt98A/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/ox-and-finch-glasgow?adjust_creative=k8b-XyMO8YirVJRAbmGY9A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=k8b-XyMO8YirVJRAbmGY9A', 'review_count': 133, 'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}, {'alias': 'scottish', 'title': 'Scottish'}], 'rating': 4.7, 'coordinates': {'latitude': 55.8656539706006, 'longitude': -4.28469195278264}, 'transactions': [], 'price': '££', 'location': {'address1': '920 Sauchiehall Street', 'address2': '', 'address3': '', 'city': 'Glasgow', 'zip_code': 'G3 7TF', 'country': 'GB', 'state': 'GLG', 'display_address': ['920 Sauchiehall Street', 'Glasgow G3 7TF', 'United Kingdom']}, 'phone': '+441413398627', 'display_phone': '+44 141 339 8627', 'distance': 1290.029875412091, 'business_

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra.uid,extra.number,extra.slots,extra.bike_uids,extra.virtual,fs_venues,yelp_venues
0,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,2025-07-01T19:18:19.376003+00:00Z,11,0,19738176,8416,6,"['47041', '823506', '46976', '46961', '46910',...",False,"[{'fsq_place_id': '4c8a3ba4a92fa093be438fbf', ...","[{'id': '_GHNOOJhhjLegiCvzf5V0Q', 'alias': 'ox..."
1,0a106cbc148d5a0c2535b51c1dbe3b4d,Parkhead,55.846524,-4.197475,2025-07-01T19:18:19.377604+00:00Z,6,0,150125231,8470,8,"['82316', '840047', '46722', '46457', '47968',...",False,"[{'fsq_place_id': '4d85e42c9324236adc05d80e', ...","[{'id': '1GgoJogpEYPisfUjmao_Ng', 'alias': 'ti..."
2,0cc8c09950e1435ee7782478ed292fef,Tollcross International Swimming Centre,55.844250,-4.176167,2025-07-01T19:18:19.376951+00:00Z,6,0,55599921,8200,6,"['828253', '46967', '823632', '828964', '82360...",False,"[{'fsq_place_id': '5c7d3478f6c04806a6b2aa4c', ...","[{'id': '3eAGIFhXXtQJ8qf0bXRx5w', 'alias': 'cu..."
3,0e94d2ad012bff0cf23497963ff5fd77,Shields Road,55.845807,-4.275232,2025-07-01T19:18:19.375677+00:00Z,1,7,7145152,8403,8,['46992'],False,"[{'fsq_place_id': '4db17f3ccda14b702a45f8c7', ...","[{'id': 'ljAyLQn_SkNY44JNl7UFlw', 'alias': 'ra..."
4,101c6cd7749f373507e9444f5222b7f2,Shawlands Shopping Centre (Kilmarnock Road),55.829057,-4.282675,2025-07-01T19:18:19.375152+00:00Z,1,6,3812605,8453,8,['46830'],False,"[{'fsq_place_id': '4b83f936f964a520191931e3', ...","[{'id': 'nhrkKcR9rjz6oYEcLuMejA', 'alias': 'or..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,f6b64fd82fe6e607f1fc788ace31c3a1,Fullarton,55.837175,-4.170209,2025-07-01T19:18:19.377474+00:00Z,3,9,114446999,8234,12,"['47334', '828994', '46917']",False,"[{'fsq_place_id': '5e049818117a3c0008354610', ...","[{'id': '0FRjuGDsa2Uaz4DT8W6kJQ', 'alias': 'mc..."
110,f806566ccd2263bd5dbb624cdd0fb7f3,Bridgeton,55.848939,-4.226823,2025-07-01T19:18:19.374456+00:00Z,7,1,349452,8441,8,"['47498', '826837', '46934', '46724', '46676',...",False,"[{'fsq_place_id': '4d938c96db1e224bbb6136cb', ...","[{'id': 'JNpi062fXUtd3lmGUSu-ww', 'alias': 'ca..."
111,f93ab2c803d81c14f7e2f1765911de04,Charing Cross Railway Station,55.864563,-4.270055,2025-07-01T19:18:19.375111+00:00Z,2,6,3456076,8421,8,"['46669', '46470']",False,"[{'fsq_place_id': '4b852d7bf964a5207d4f31e3', ...","[{'id': '1kVFS_JFcNKO2-twa2lOnQ', 'alias': 'tw..."
112,fa6bc682e8627d66891c2b6250c81b1e,Mitchell Library,55.864788,-4.271852,2025-07-01T19:18:19.374157+00:00Z,4,2,264308,8425,6,"['47634', '828248', '46535', '46508']",False,"[{'fsq_place_id': '4b715b5df964a5209a422de3', ...","[{'id': '1kVFS_JFcNKO2-twa2lOnQ', 'alias': 'tw..."


In [ ]:
# A view of the yelp_venues data saved in the city_bikes dataframe
city_bikes_df['yelp_venues'][0]

[{'id': '_GHNOOJhhjLegiCvzf5V0Q',
  'alias': 'ox-and-finch-glasgow',
  'name': 'Ox And Finch',
  'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/nro4UcJrJkLiPOQ8Yvt98A/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/ox-and-finch-glasgow?adjust_creative=k8b-XyMO8YirVJRAbmGY9A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=k8b-XyMO8YirVJRAbmGY9A',
  'review_count': 133,
  'categories': [{'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
   {'alias': 'scottish', 'title': 'Scottish'}],
  'rating': 4.7,
  'coordinates': {'latitude': 55.8656539706006,
   'longitude': -4.28469195278264},
  'transactions': [],
  'price': '££',
  'location': {'address1': '920 Sauchiehall Street',
   'address2': '',
   'address3': '',
   'city': 'Glasgow',
   'zip_code': 'G3 7TF',
   'country': 'GB',
   'state': 'GLG',
   'display_address': ['920 Sauchiehall Street',
    'Glasgow G3 7TF',
    'United Kingdom']},
  'phone': '+441413398627',
  'display_phone'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [23]:
# flatten the json returned by the Yelp API and stored in the city_bikes_df dataframe in the column 'yelp_venues'
yelp_venues_df = pd.json_normalize(
    city_bikes_df['yelp_venues'][0],
    record_path="categories",       # Path to expand nested lists
    meta=["id", "name", "rating",["coordinates","latitude"],["coordinates","longitude"],["location", "address1"], ["location", "city"], "distance"],  # Top-level columns
    meta_prefix="business_",
    record_prefix="category_",
    errors = 'ignore'
)
yelp_venues_df


,category_alias,category_title,business_id,business_name,business_rating,business_coordinates.latitude,business_coordinates.longitude,business_location.address1,business_location.city,business_distance
0,tapasmallplates,Tapas/Small Plates,_GHNOOJhhjLegiCvzf5V0Q,Ox And Finch,4.7,55.865654,-4.284692,920 Sauchiehall Street,Glasgow,1290.029875
1,scottish,Scottish,_GHNOOJhhjLegiCvzf5V0Q,Ox And Finch,4.7,55.865654,-4.284692,920 Sauchiehall Street,Glasgow,1290.029875
2,british,British,jGLzc9eIUAnfto_5oZ8BMQ,Number 16 Restaurant,4.6,55.87081,-4.298839,16 Byres Road,Glasgow,700.694335
3,bars,Bars,rmaH1My396rCYEnL8XQtPg,Ubiquitous Chip,4.3,55.874915,-4.293247,8-12 Ashton Lane,Glasgow,1274.920333
4,scottish,Scottish,rmaH1My396rCYEnL8XQtPg,Ubiquitous Chip,4.3,55.874915,-4.293247,8-12 Ashton Lane,Glasgow,1274.920333
5,british,British,rmaH1My396rCYEnL8XQtPg,Ubiquitous Chip,4.3,55.874915,-4.293247,8-12 Ashton Lane,Glasgow,1274.920333
6,steak,Steakhouses,hTYpLa8-w4WdCzyRjS5z5A,The Butchershop Bar & Grill,4.4,55.867009,-4.289232,1055 Sauchiehall Street,Glasgow,1017.470608
7,wine_bars,Wine Bars,hTYpLa8-w4WdCzyRjS5z5A,The Butchershop Bar & Grill,4.4,55.867009,-4.289232,1055 Sauchiehall Street,Glasgow,1017.470608
8,seafood,Seafood,l8UWRblDIfIHWMl5jOd-4Q,Crabshakk,4.5,55.865014,-4.284668,1114 Argyle Street,Glasgow,1285.19906
9,indpak,Indian,XutHqBQvVgrCqHrxIgsgrQ,Mother India's Café,4.5,55.867383,-4.291093,1355 Argyle Street,Glasgow,910.450582


In [24]:
yelp_venues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   category_alias                  32 non-null     object
 1   category_title                  32 non-null     object
 2   business_id                     32 non-null     object
 3   business_name                   32 non-null     object
 4   business_rating                 32 non-null     object
 5   business_coordinates.latitude   32 non-null     object
 6   business_coordinates.longitude  32 non-null     object
 7   business_location.address1      32 non-null     object
 8   business_location.city          32 non-null     object
 9   business_distance               32 non-null     object
dtypes: object(10)
memory usage: 2.6+ KB


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

*Answer*: The Yelp API appears to give more complete venue information. Ratings were easy to find and process. Yelp also gives opening hours, links to things like menus, etc. By contrast, the FourSquare API seemed to be about its internal classifications. The classification information was prominent.

Get the top 10 restaurants according to their rating

In [ ]:
# The top-rated venues on Yelp, returned by the city bikes locations
# (I couldn't find similar ratings in the FourSquare response)
yelp_venues_df.sort_values(by='business_rating', ascending=False).head(10)

,category_alias,category_title,business_id,business_name,business_rating,business_coordinates.latitude,business_coordinates.longitude,business_location.address1,business_location.city
0,tapasmallplates,Tapas/Small Plates,_GHNOOJhhjLegiCvzf5V0Q,Ox And Finch,4.7,55.865654,-4.284692,920 Sauchiehall Street,Glasgow
10,scottish,Scottish,8Mpp1Pc17OcuiVyeaFBdeA,The Gannet,4.7,55.864975,-4.285031,1155 Argyle Street,Glasgow
1,scottish,Scottish,_GHNOOJhhjLegiCvzf5V0Q,Ox And Finch,4.7,55.865654,-4.284692,920 Sauchiehall Street,Glasgow
11,bistros,Bistros,8Mpp1Pc17OcuiVyeaFBdeA,The Gannet,4.7,55.864975,-4.285031,1155 Argyle Street,Glasgow
31,indpak,Indian,voOe91fmlsldhbHlHrQwBQ,Horn Please,4.7,55.865776,-4.284446,914-916 Sauchiehall Street,Glasgow
28,british,British,R1NcLn3Qi3un6LcaNiPrjQ,Cail Bruich West,4.6,55.877485,-4.289126,725 Great Western Road,Glasgow
27,scottish,Scottish,R1NcLn3Qi3un6LcaNiPrjQ,Cail Bruich West,4.6,55.877485,-4.289126,725 Great Western Road,Glasgow
2,british,British,jGLzc9eIUAnfto_5oZ8BMQ,Number 16 Restaurant,4.6,55.87081,-4.298839,16 Byres Road,Glasgow
17,steak,Steakhouses,7DFVBsQXKuIrrj0-xl17Ew,Porter & Rye,4.6,55.864676,-4.284431,1131 Argyle Street,Glasgow
16,indpak,Indian,mO4Tn8uMPKgbM0RjOWgtKg,Mother India,4.5,55.865299,-4.283917,28 Westminster Terrace,Glasgow
